In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [5]:
from ftnmr import *

In [6]:
molecules = []

Hydrogens and their frequencies and ratios

In [16]:
b0 = 0.30
b1 = 0.30001

In [17]:
np.isclose(b0, b1)

False

In [66]:
from string import ascii_lowercase as asc

In [18]:
from scipy.special import binom

In [81]:
A = {
    'a':(1, 2.3),
    'b':(1, 3.3),
    'c':(3, 4.5)}

j1 = ('a', 'b', 3.200)
j2 = ('b', 'c', 3.200)
B = [j1, j2]

In [88]:
C = {k:[ (b[2], A[ b[ 1-b.index(k) ] ][0]) for b in B if k in b] for k in A}

In [87]:
C = {a:[] for a in A}
print(C)

for k in A:
    for b in B:
        if k == b[0]: C[k].append( (b[2], A[b[1]][0]) )
        elif k == b[1]: C[k].append( (b[2], A[b[0]][0]) )

C

{'a': [], 'b': [], 'c': []}


{'a': [(3.2, 1)], 'b': [(3.2, 1), (3.2, 3)], 'c': [(3.2, 1)]}

In [94]:
F0 = {k:[ [n*b/2 for n in range(-d, d+1, 2)] for b, d in C[k]] for k in A}
F1 = {k:[ [binom(d, x)/pow(2,d) for x in range(0, d+1)] for b, d in C[k]] for k in A}
F = {k:(F0[k], F1[k]) for k in A}
F

{'a': ([[-1.6, 1.6]], [[0.5, 0.5]]),
 'b': ([[-1.6, 1.6], [-4.800000000000001, -1.6, 1.6, 4.800000000000001]],
  [[0.5, 0.5], [0.125, 0.375, 0.375, 0.125]]),
 'c': ([[-1.6, 1.6]], [[0.5, 0.5]])}

In [93]:
F = {a:([], []) for a in A}
for k in A:
    for b, d in C[k]:
        F[k][0].append( np.array([n*b/2 for n in range(-d, d+1, 2)]) )
        F[k][1].append( np.array([binom(d, x)/pow(2,d) for x in range(0, d+1)]))

F

{'a': ([array([-1.6,  1.6])], [array([0.5, 0.5])]),
 'b': ([array([-1.6,  1.6]), array([-4.8, -1.6,  1.6,  4.8])],
  [array([0.5, 0.5]), array([0.125, 0.375, 0.375, 0.125])]),
 'c': ([array([-1.6,  1.6])], [array([0.5, 0.5])])}

In [23]:
# this combines the above two cells into one

F = {a:([], []) for a in A}

for k in A:
    for b in B:
        if k == b[0]: 
            d = A[b[1]][0]
            F[k][0].append( [n*b[2]/2 for n in range(-d, d+1, 2)] )
            F[k][1].append( [binom(d, x)/pow(2,d) for x in range(0, d+1)] )
        elif k == b[1]: 
            d = A[b[0]][0]
            F[k][0].append( [n*b[2]/2 for n in range(-d, d+1, 2)] )
            F[k][1].append( [binom(d, x)/pow(2,d) for x in range(0, d+1)] )

F

{'a': ([[-1.6, 1.6]], [[0.5, 0.5]]),
 'b': ([[-1.6, 1.6], [-4.800000000000001, -1.6, 1.6, 4.800000000000001]],
  [[0.5, 0.5], [0.125, 0.375, 0.375, 0.125]]),
 'c': ([[-1.6, 1.6]], [[0.5, 0.5]])}

In [26]:
from itertools import product

In [101]:
G0 = {k:[ sum(i) for i in product(*F0[k]) ] for k in A}
G1 = {k:[ np.prod(i) for i in product(*F1[k]) ] for k in A}
H = {k:(G0[k], G1[k]) for k in A}
H

{'a': ([-1.6, 1.6], [0.5, 0.5]),
 'b': ([-6.4,
   -3.2,
   0.0,
   3.2000000000000006,
   -3.2000000000000006,
   0.0,
   3.2,
   6.4],
  [0.0625, 0.1875, 0.1875, 0.0625, 0.0625, 0.1875, 0.1875, 0.0625]),
 'c': ([-1.6, 1.6], [0.5, 0.5])}

In [102]:
G = {a:([], []) for a in A}

for k in A:
    for i in product(*F[k][0]):
        G[k][0].append(sum(i))
        
    for i in product(*F[k][1]):
        G[k][1].append(np.prod(i))

G

{'a': ([-1.6, 1.6], [0.5, 0.5]),
 'b': ([-6.4,
   -3.2,
   0.0,
   3.2000000000000006,
   -3.2000000000000006,
   0.0,
   3.2,
   6.4],
  [0.0625, 0.1875, 0.1875, 0.0625, 0.0625, 0.1875, 0.1875, 0.0625]),
 'c': ([-1.6, 1.6], [0.5, 0.5])}

In [103]:
G == H

True

In [116]:
J0 = {k:[G0[k][n] for n in np.argsort(G0[k])] for k in A}
J1 = {k:[G1[k][n] for n in np.argsort(G0[k])] for k in A}
J = {k:(J0[k], J1[k]) for k in A}
J

{'a': ([-1.6, 1.6], [0.5, 0.5]),
 'b': ([-6.4,
   -3.2000000000000006,
   -3.2,
   0.0,
   0.0,
   3.2,
   3.2000000000000006,
   6.4],
  [0.0625, 0.0625, 0.1875, 0.1875, 0.1875, 0.1875, 0.0625, 0.0625]),
 'c': ([-1.6, 1.6], [0.5, 0.5])}

In [117]:
I = {k:([],[]) for k in A}

for k in A:
    for n in np.argsort(H[k][0]):
        I[k][0].append(H[k][0][n])
        I[k][1].append(H[k][1][n])

I

{'a': ([-1.6, 1.6], [0.5, 0.5]),
 'b': ([-6.4,
   -3.2000000000000006,
   -3.2,
   0.0,
   0.0,
   3.2,
   3.2000000000000006,
   6.4],
  [0.0625, 0.0625, 0.1875, 0.1875, 0.1875, 0.1875, 0.0625, 0.0625]),
 'c': ([-1.6, 1.6], [0.5, 0.5])}

In [118]:
J == I

True

In [113]:
np.isclose(1.23452, 1.23453)

True

In [ ]:
K0 = {k:[ for i in range(1, len(J0[k])) if not np.isclose(, J0[k])] for k in A}
K1 = {k:[ for i in range(1, len(J0[k])) if not np.isclose(, J0[k])] for k in A}

In [122]:
K = {k:([J0[k][0]],[J1[k][0]]) for k in A}

for k in A:
    for i in range(1, len(J[k][0])):
        if np.isclose(K[k][0][-1], J[k][0][i]):
            K[k][1][-1] += J[k][1][i]
        else:
            K[k][0].append(J[k][0][i])
            K[k][1].append(J[k][1][i])

In [123]:
K

{'a': ([-1.6, 1.6], [0.5, 0.5]),
 'b': ([-6.4, -3.2000000000000006, 0.0, 3.2, 6.4],
  [0.0625, 0.25, 0.375, 0.25, 0.0625]),
 'c': ([-1.6, 1.6], [0.5, 0.5])}

In [109]:
sum(K['b'][1])

1.0

Molecule construction

In [213]:
a = ('a', 6, 2.3)
b = ('b', 4, 3.4)
c = ('c', 2, 4.5)
A = [a, b, c]

j1 = ('a', 'b', 3.300)
j2 = ('b', 'c', 3.000)
B = [j1, j2]

In [222]:
class molecule():
    def __init__(
        self,
        A: list,
        B: list):
        self.A = A
        self.B = B

In [223]:
a = molecule(A=[1, 2, 3], B=2)

In [230]:
def add(x: int, y: int) -> int:
    return x + y

In [231]:
add(2, 4)

6

In [234]:
num: str = add('bab', 'o')

In [235]:
num

'babo'